In [1]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union
from pydantic.config import ConfigDict

from src.context import context

from src.utilities import *
from src.bubble import *

import instructor



Retrieved company Darty : 1707313014508x102198350946437700
Retrieved project Darty_trustpilot : 1707329196900x870734705097005300


In [3]:
aspects_df = get("Aspect")

In [4]:
aspects_df.groupby()

,Created Date,Created By,Modified Date,Category,Company,Project,Rating,Sub_category,Associated_feedback,Date,_id,Explanation
0,2024-02-16 11:27:12.191000+00:00,admin_user_feedback-analysis_test,2024-02-16 11:27:12.191000+00:00,1708015555422x783692141072623600,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1708015555741x283185418299470850,1707643040311x281208184060071520,2024-02-07T00:00:00.000Z,1708082832191x432611259722250700,NaN
1,2024-02-16 11:27:12.196000+00:00,admin_user_feedback-analysis_test,2024-02-16 11:27:12.196000+00:00,1708015553720x307677447830442240,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1708015554529x444035970008549250,1707643040311x281208184060071520,2024-02-07T00:00:00.000Z,1708082832196x169167127065831040,NaN
2,2024-02-16 11:27:12.201000+00:00,admin_user_feedback-analysis_test,2024-02-16 11:27:12.201000+00:00,1708015570991x523003420569127600,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1708015571499x209656851322028060,1707643040311x281208184060071520,2024-02-07T00:00:00.000Z,1708082832201x424770499809504500,Le personnel est très accueillant et sympathique.
3,2024-02-16 11:27:12.204000+00:00,admin_user_feedback-analysis_test,2024-02-16 11:27:12.204000+00:00,1708015553720x307677447830442240,1707313014508x102198350946437700,1707329196900x870734705097005300,4,1708015554131x995344137291493100,1707643040311x281208184060071520,2024-02-07T00:00:00.000Z,1708082832204x100311638374955820,NaN
4,2024-02-16 11:27:14.160000+00:00,admin_user_feedback-analysis_test,2024-02-16 11:27:14.160000+00:00,1708015569716x598042261554836900,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1708015570097x862705649430030700,1707643040316x844188021545895200,2024-01-31T00:00:00.000Z,1708082834160x545020796352881500,La prise en charge de l'ancienne machine et l'...
...,...,...,...,...,...,...,...,...,...,...,...,...
192,2024-02-16 13:31:09.778000+00:00,admin_user_feedback-analysis_test,2024-02-16 13:31:09.778000+00:00,1708015577252x623767797641778300,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1708015577555x688807197740501000,1707643041098x736616143542145000,2024-01-22T00:00:00.000Z,1708090269778x835079627609417500,NaN
193,2024-02-16 13:31:09.784000+00:00,admin_user_feedback-analysis_test,2024-02-16 13:31:09.784000+00:00,1708015577252x623767797641778300,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1708015577923x774632487944991200,1707643041098x736616143542145000,2024-01-22T00:00:00.000Z,1708090269784x428030243785141950,NaN
194,2024-02-16 13:31:10.885000+00:00,admin_user_feedback-analysis_test,2024-02-16 13:31:10.886000+00:00,1708015577252x623767797641778300,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1708015577555x688807197740501000,1707643041101x520551676662069500,2024-01-26T00:00:00.000Z,1708090270885x494323147138308300,J'ai rapidement trouvé un conseil adapté à mon...
195,2024-02-16 13:31:10.896000+00:00,admin_user_feedback-analysis_test,2024-02-16 13:31:10.896000+00:00,1708015577252x623767797641778300,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1708015580113x761173827728466800,1707643041101x520551676662069500,2024-01-26T00:00:00.000Z,1708090270896x425548126484849100,NaN
